# 01 - Ingestão

Verifica a presença do vídeo no DBFS.

In [0]:
import os
# Install dependencies in the notebook environment (optional - uncomment if running in fresh environment)
# !python -m pip install -r requirements.txt

from src.config import load_config
cfg = load_config()
video_dbfs = cfg.get('video_path')
print('Existe?', os.path.exists(video_dbfs) if video_dbfs else False)
print('Se não existir, faça upload manual do vídeo para /FileStore/videos/ ou atualize configs/config.yaml')


In [ ]:
# Opcional: Gera pseudo-rótulos automaticamente antes do pré-processamento
import os
import sys
from src.config import load_config
cfg = load_config()
video = cfg.get('video_path')
frames_dir = cfg.get('frames_dir')
ann_path = os.path.join(frames_dir, 'annotations.json')

if not video or not frames_dir:
    print('Configuração incompleta: verifique video_path e frames_dir em configs/config.yaml')
else:
    # garantir que o diretório de scripts esteja no path
    sys.path.append(os.path.abspath('.'))
    try:
        from scripts.pseudo_label import pseudo_label
    except Exception as e:
        print('Não foi possível importar pseudo_label:', e)
        raise

    # se quiser pular o passo, defina SKIP_PSEUDO=1 no ambiente
    if os.environ.get('SKIP_PSEUDO') == '1':
        print('Pseudo-labeling pulado via SKIP_PSEUDO=1')
    else:
        print('Iniciando pseudo-labeling — pode demorar dependendo do modelo/video...')
        # Force DeepSORT usage; set use_deepsort=False to force fallback
        pseudo_label(video, frames_dir, ann_path, sample_rate=cfg.get('sample_rate', 2))
        print('Pseudo-labeling finalizado. Anotações:', ann_path)

        # Mostrar alguns frames anotados inline (se rodando em notebook)
        try:
            from IPython.display import display, Image
            annotated_dir = os.path.join(frames_dir, 'annotated')
            imgs = sorted([os.path.join(annotated_dir, f) for f in os.listdir(annotated_dir) if f.endswith('.jpg')])[:5]
            for p in imgs:
                display(Image(filename=p))
        except Exception:
            pass


# 02 - Pré-processamento

Extrai frames e salva em `/dbfs/FileStore/frames/`.

In [0]:
from src.data.extract_frames import extract_frames
from src.config import load_config
import os
cfg = load_config()
VIDEO = cfg.get('video_path')
OUT_DIR = cfg.get('frames_dir')
os.makedirs(OUT_DIR, exist_ok=True)
n = extract_frames(VIDEO, OUT_DIR, sample_rate=cfg.get('sample_rate', 2))
print('Frames extraídos:', n)


# 03 - Treino

Treina um modelo simples com imagens extraídas.

In [0]:
from src.models.train import train_from_folder
from src.config import load_config
import os

cfg = load_config()
DATA_DIR = cfg.get('frames_dir')
MODEL_DIR = cfg.get('model_dir')
os.makedirs(MODEL_DIR, exist_ok=True)
train_from_folder(DATA_DIR, MODEL_DIR, epochs=1)

# 04 - Avaliação

Avalia o modelo salvo (exemplo rápido).

In [0]:
from src.models.train import evaluate_folder
from src.config import load_config
cfg = load_config()
MODEL_DIR = cfg.get('model_dir')
DATA_DIR = cfg.get('frames_dir')
evaluate_folder(MODEL_DIR, DATA_DIR)


# 05 - Serving

Exemplo de inferência local usando o modelo salvo.

In [0]:
from src.serving.serve_inference import infer_image
from src.config import load_config
import os
cfg = load_config()
MODEL_PATH = os.path.join(cfg.get('model_dir',''), 'model.pth')
TEST_IMAGE = os.path.join(cfg.get('frames_dir',''), 'frame_00000.jpg')
print(infer_image(MODEL_PATH, TEST_IMAGE))
